In [25]:
!pip install groq python-dotenv


In [26]:
import os
import getpass
from groq import Groq

# Securely input API key
os.environ["GROQ_API_KEY"] = getpass.getpass("YOUR_GROQ_API_KEY")

client = Groq(api_key=os.environ["GROQ_API_KEY"])



YOUR_GROQ_API_KEY··········


In [27]:
MODEL_NAME = "llama-3.1-8b-instant"



MODEL_CONFIG = {
    "technical": {
        "system_prompt": (
            "You are a technical support expert. "
            "Be precise, logical, and code-focused. "
            "Provide debugging steps and short code examples when relevant."
        ),
        "temperature": 0.7
    },
    "billing": {
        "system_prompt": (
            "You are a billing and account support specialist. "
            "Be empathetic, clear, and policy-driven. "
            "Explain refunds, charges, and subscriptions politely."
        ),
        "temperature": 0.7
    },
    "general": {
        "system_prompt": (
            "You are a friendly general customer support assistant. "
            "Answer casually and helpfully."
        ),
        "temperature": 0.7
    }
}

In [28]:
def route_prompt(user_input: str) -> str:
    router_system_prompt = (
        "You are a routing assistant. "
        "Classify this text into one of these categories: "
        "[technical, billing, general]. "
        "Return ONLY the category word."
    )

    response = client.chat.completions.create(
        model=MODEL_NAME,
        temperature=0,
        messages=[
            {"role": "system", "content": router_system_prompt},
            {"role": "user", "content": user_input}
        ]
    )

    category = response.choices[0].message.content.strip().lower()
    return category


In [29]:
def process_request(user_input: str) -> str:
    # Step 1: Route
    category = route_prompt(user_input)

    # Step 2: Safety fallback
    if category not in MODEL_CONFIG:
        category = "general"

    config = MODEL_CONFIG[category]

    # Step 3: Call expert
    response = client.chat.completions.create(
        model=MODEL_NAME,
        temperature=config["temperature"],
        messages=[
            {"role": "system", "content": config["system_prompt"]},
            {"role": "user", "content": user_input}
        ]
    )

    answer = response.choices[0].message.content
    return f"Routed to: {category}\n\nResponse:\n{answer}"


In [30]:
test_queries = [
    "My python script is throwing an IndexError on line 5.",
    "I was charged twice for my subscription this month.",
    "Hey, what features do you offer?"
]

for q in test_queries:
    print("User:", q)
    print(process_request(q))
    print("-" * 50)

User: My python script is throwing an IndexError on line 5.
Routed to: technical

Response:
An `IndexError` typically occurs when you're trying to access an element in a list or other sequence that doesn't exist. Here are some general steps to troubleshoot the issue:

1. **Check the length of the list**: Make sure the list you're trying to access has at least as many elements as you're trying to access. You can do this by printing out the length of the list before the line that's causing the error.

   ```python
print(len(your_list))
```

2. **Check the index**: Double-check that the index you're using is within the valid range. In Python, indices start at 0, so if your list has 5 elements, the valid indices are 0, 1, 2, 3, and 4.

   ```python
index = 5
if index >= len(your_list):
    print("Index out of range")
```

3. **Check for empty lists**: If you're trying to access an element in a list that might be empty, you should check for that condition before trying to access the element

check for the available free models on groq , we use any of these available models

In [31]:
for m in client.models.list().data:
    print(m.id)


llama-3.1-8b-instant
meta-llama/llama-4-maverick-17b-128e-instruct
canopylabs/orpheus-v1-english
openai/gpt-oss-120b
qwen/qwen3-32b
llama-3.3-70b-versatile
moonshotai/kimi-k2-instruct-0905
whisper-large-v3-turbo
allam-2-7b
meta-llama/llama-prompt-guard-2-86m
groq/compound-mini
openai/gpt-oss-safeguard-20b
openai/gpt-oss-20b
whisper-large-v3
canopylabs/orpheus-arabic-saudi
groq/compound
meta-llama/llama-guard-4-12b
meta-llama/llama-prompt-guard-2-22m
meta-llama/llama-4-scout-17b-16e-instruct
moonshotai/kimi-k2-instruct


BONUS TOOL

In [32]:
def get_bitcoin_price():
    return "Bitcoin is currently around $65,000 (mock data)."


In [33]:
def route_prompt(user_input: str) -> str:
    router_system_prompt = (
        "You are a routing assistant. "
        "Classify this text into one of these categories: "
        "[technical, billing, general, tool]. "
        "Use 'tool' if the user is asking for live data like prices or weather. "
        "Return ONLY the category word."
    )

    response = client.chat.completions.create(
        model=MODEL_NAME,
        temperature=0,
        messages=[
            {"role": "system", "content": router_system_prompt},
            {"role": "user", "content": user_input}
        ]
    )

    return response.choices[0].message.content.strip().lower()


In [34]:
def process_request(user_input: str) -> str:
    category = route_prompt(user_input)

    if category == "tool":
        return f"Routed to: tool\n\nResponse:\n{get_bitcoin_price()}"

    if category not in MODEL_CONFIG:
        category = "general"

    config = MODEL_CONFIG[category]

    response = client.chat.completions.create(
        model=MODEL_NAME,
        temperature=config["temperature"],
        messages=[
            {"role": "system", "content": config["system_prompt"]},
            {"role": "user", "content": user_input}
        ]
    )

    answer = response.choices[0].message.content
    return f"Routed to: {category}\n\nResponse:\n{answer}"


In [35]:
print(process_request("What is the current price of Bitcoin?"))


Routed to: tool

Response:
Bitcoin is currently around $65,000 (mock data).
